### Imports

In [2]:
import pandas as pd
import pickle
import os
import sys
sys.path.append('/Users/shashankkumar/Documents/GitHub/MacroEcon')
from models.macro_economics import preprocess_data

File Paths

In [5]:
INPUT_DIR_NZ = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NZ/input_pop_data"
INPUT_DIR_NYC = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/input_pop_data"

OUTPUT_DIR_NZ = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NZ/output_pop_data"
OUTPUT_DIR_NYC = "/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/output_pop_data"

if not os.path.exists(OUTPUT_DIR_NZ):
    os.mkdir(OUTPUT_DIR_NZ)

if not os.path.exists(OUTPUT_DIR_NYC):
    os.mkdir(OUTPUT_DIR_NYC)

### NZ

In [ ]:
household_file = os.path.join(INPUT_DIR_NZ, "household.pickle")
df = pd.read_pickle(household_file)
df_household = df['household']

In [ ]:
grouped_households_df = df_household.groupby('area')[['people_num', 'children_num', 'household_num']].sum()
grouped_households_df["Family_households"] = grouped_households_df['children_num'] != 0
grouped_households_df['Family_households'] = grouped_households_df['Family_households'].astype(int)
grouped_households_df['Family_households'] = grouped_households_df['Family_households'].astype(int)
grouped_households_df['Nonfamily_households'] = 1 - grouped_households_df['Family_households']
grouped_households_df

Save household data

In [ ]:
NZ_HOUEHOLD_OUT = os.path.join(OUTPUT_DIR_NZ, "household.pkl")
grouped_households_df.to_pickle(NZ_HOUEHOLD_OUT)

Age Gender Ethnicity

In [ ]:
age_gender_file = os.path.join(INPUT_DIR_NZ, "population.pickle")
df = pd.read_pickle(age_gender_file)
df_age = df['age']
df_age_gender = df['gender']
df_ethnicity = df['ethnicity']
age_list = ['U19','20t29', '30t39', '40t49', '50t64', '65A']

Age-gender

In [ ]:
df_age_gender_combined = pd.DataFrame()
df_age_gender_combined['U19'] = df_age_gender.loc[:, 0:19].sum(axis=1)
df_age_gender_combined['20t29'] = df_age_gender.loc[:, 20:29].sum(axis=1)
df_age_gender_combined['30t39'] = df_age_gender.loc[:, 30:39].sum(axis=1)
df_age_gender_combined['40t49'] = df_age_gender.loc[:, 40:49].sum(axis=1)
df_age_gender_combined['50t64'] = df_age_gender.loc[:, 50:64].sum(axis=1)
df_age_gender_combined['65A'] = df_age_gender.loc[:, 65:100].sum(axis=1)
df_age_gender_combined['area'] = df_age_gender['area']
df_age_gender_combined['gender'] = df_age_gender['gender']
df_age_gender_combined = df_age_gender_combined.melt(
        id_vars=["area", "gender"], var_name="age", value_name="count"
    )
df_age_gender_combined

Ethnicity

In [ ]:
df_ethnicity_combined = pd.DataFrame()
df_ethnicity_combined['count']  = df_ethnicity.loc[:, 0:100].sum(axis=1)
df_ethnicity_combined['ethnicity']  = df_ethnicity['ethnicity']
df_ethnicity_combined['area'] = df_ethnicity['area']
df_ethnicity_combined

Save the data

In [ ]:
NZ_POP_OUT = os.path.join(OUTPUT_DIR_NZ, "NZ_POP.pkl")
att_NZ = {
    'age_gender':df_age_gender_combined,
    'ethnicity':df_ethnicity_combined
}
with open(NZ_POP_OUT, 'wb') as f:
    pickle.dump(att_NZ, f)

### NYC

NYC Data from pickle file saved by agents_nta.py

In [6]:
NYC_DATA_PKL = os.path.join(INPUT_DIR_NYC, "output_dict.pkl")
dfn = pd.read_pickle(NYC_DATA_PKL)

Gender

In [7]:
dfa = dfn['age_gender']
df_gender_melt = dfa.melt(
        id_vars=["area", "gender"], var_name="age", value_name="count"
    )
df_gender_melt

,area,gender,age,count
0,BK0101,female,20t29,3396
1,BK0101,male,20t29,3327
2,BK0102,female,20t29,7410
3,BK0102,male,20t29,6867
4,BK0103,female,20t29,2480
...,...,...,...,...
3139,SI9591,male,U19,0
3140,SI9592,female,U19,0
3141,SI9592,male,U19,28
3142,SI9593,female,U19,0


Ethnicity

In [8]:
# Melt the DataFrame
dfe = dfn['ethnicity']
dfe_melted = dfe.melt(id_vars='area', var_name='ethnicity', value_name='count')

# Sort the DataFrame by 'area' and 'ethnicity'
dfe_melted.sort_values(['area', 'ethnicity'], inplace=True)

# Reset the index of the melted DataFrame
dfe_melted.reset_index(drop=True, inplace=True)
dfe_melted

,area,ethnicity,count
0,BK0101,asian,1464
1,BK0101,black,937
2,BK0101,hispanic,5498
3,BK0101,native,22
4,BK0101,other,2324
...,...,...,...
1567,SI9593,black,0
1568,SI9593,hispanic,0
1569,SI9593,native,0
1570,SI9593,other,0


Save NYC data

In [9]:
NYC_POP_OUT = os.path.join(OUTPUT_DIR_NYC, "NYC_POP.pkl")
att_dict = {'age_gender' : df_gender_melt,
    'ethnicity' : dfe_melted,}
with open(NYC_POP_OUT, 'wb') as f:
    pickle.dump(att_dict, f)

Load and test

In [10]:
dfat = pd.read_pickle("/Users/shashankkumar/Documents/GitHub/MacroEcon/simulator_data/NYC/output_pop_data/NYC_POP.pkl")
dfat

{'age_gender':         area  gender    age  count
 0     BK0101  female  20t29   3396
 1     BK0101    male  20t29   3327
 2     BK0102  female  20t29   7410
 3     BK0102    male  20t29   6867
 4     BK0103  female  20t29   2480
 ...      ...     ...    ...    ...
 3139  SI9591    male    U19      0
 3140  SI9592  female    U19      0
 3141  SI9592    male    U19     28
 3142  SI9593  female    U19      0
 3143  SI9593    male    U19      0
 
 [3144 rows x 4 columns],
 'ethnicity':         area ethnicity  count
 0     BK0101     asian   1464
 1     BK0101     black    937
 2     BK0101  hispanic   5498
 3     BK0101    native     22
 4     BK0101     other   2324
 ...      ...       ...    ...
 1567  SI9593     black      0
 1568  SI9593  hispanic      0
 1569  SI9593    native      0
 1570  SI9593     other      0
 1571  SI9593     white      0
 
 [1572 rows x 3 columns]}

Household

In [ ]:
mapping = {"HHPopE": "Population_in_households",
"RshpHHldrE": "Householder",
"Rshp_SpE": "Spouse",
"Rshp_ChE": "children_num",
"Rshp_OthRE": "Other_relatives",
"Rshp_NRE": "Nonrelatives",
"Rshp_NRUPE": "Unmarried_partner",
"Rshp_SSUPE": "Same_sex_unmarried_partner",
"Fam1E": "Family_households",
"FamChU18E": "With_own_children_under_18_years",
"MrdFamE": "Married_couple_family",
"MrdChU18E": "With_own_children_under_18_years",
"MHnSE": "Male_householder_no spouse_present_family",
"MHnSChU18E": "With_own_children_under_18_years",
"FHnSE": "Female_householder_no_spouse_present_family",
"FHnSChU18E": "With_own_children_under_18_years",
"NFam1E": "Nonfamily_households",
"NFamAE": "Householder_living alone",
"NFamA65plE": "65_years_and_over",
'GeoID':'area', 
'HHPopE':'people_num', 
'HH1E':'household_num',
"AvgHHSzE" : 'Average_household_size',
"AvgFmSzE" : 'Average_family_size'
}

In [ ]:
df_housing = pd.read_excel('/Users/shashankkumar/Documents/AgentTorch_Official/AgentTorch/AgentTorch/helpers/census_data/nyc/nta/soc_2021acs5yr_nta.xlsx')
attributes = list(mapping.keys())
df_housing = df_housing[attributes]
df_housing.rename(columns=mapping, inplace=True)
df_housing

In [ ]:
NYC_HOUSEHOLD_OUT = os.path.join(OUTPUT_DIR_NYC, "household.pkl")
df_housing.to_pickle(NYC_HOUSEHOLD_OUT)

### Extra Data can be used later, Ignore for now

In [ ]:
dfed = dfn['education']

In [ ]:
# Melt the DataFrame
df_melted = dfed.melt(id_vars='area', var_name='education', value_name='total_population')

# Sort the DataFrame by 'area' and 'ethnicity'
df_melted.sort_values(['area', 'education'], inplace=True)

# Reset the index of the melted DataFrame
df_melted.reset_index(drop=True, inplace=True)
df_melted

In [ ]:
dfei = dfn['employment_insurance']

In [ ]:
# Melt the DataFrame
df_melted = dfei.melt(id_vars='area', var_name='employment_insurance', value_name='total_population')

# Sort the DataFrame by 'area' and 'ethnicity'
df_melted.sort_values(['area', 'employment_insurance'], inplace=True)

# Reset the index of the melted DataFrame
df_melted.reset_index(drop=True, inplace=True)
df_melted